# AutoKeras AutoCV para Regressão de Imagens - Experimento

Este componente utiliza [AutoKeras](https://autokeras.com/) AutoCV para a **tarefa de regressão**.

O algoritmo faz a busca por arquiteturas e hyperparâmetros que melhor configuram o modelo
para a base de dados fornecida.

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

Para esse componente, a base de dados deve estar no seguinte formado:
- Imagens coloridas (3 canais) no formato 256x256 pixels. Caso não estejam nesse formato, o código faz as alterações necesssárias
- Cada classe tem sua pasta com suas respectivas imagens, além dos conjuntos de treino, validação e teste terem suas pastas separadas. Um exemplo da árvore de diretórios pode ser observado abaixo:

```bash
dataset
|________train
|        |_____class_name1
|        |     |____image0.jpg
|        |     |____image1.jpg
|        |     ...
|        |
|        |_____class_name2
|              |____image3.jpg
|              |____image4.jpg
|               ...
|________test
|        |_____class_name1
|        |     |____image9.jpg
|        |     |____image10.jpg
|        |     ...
|        |
|        |_____class_name2
|              |____image11.jpg
|              |____image12.jpg
|              ...
```

In [ ]:
dataset = "" #@param {type:"string"}
num_epochs = 16 # Número de épocas.
trials = 5 # Número de tentativas para o algoritmo de busca por arquiteturas e hyperparametros.
batch_size = 8 # Tamanho do lote de imagens
target_size = (256, 256) # Tamanho das imagens de entrada da rede
validation = 0.2 # Número entre [0, 1] que representa a porcentagem de dados de treino que será usada para validação


'''
[OPCIONAIS]
Aumentações são técnicas que comprovadamente ajudam 
a melhorar o desempenho e generalização dos modelos.
Essas técnicas serão utilizadas apenas nos conjuntos de treino e validação. 
Não utilizadas no conjunto de teste.
Possibilidades de aumentações para utilizar no dataset estão listadas abaixo:

Mais explicações sobre cada aumentação podem ser encontradas no link:
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
'''

brightness_range = None # Tupla ou lista de dois float. Faixa para escolher o valor da mudança de brilho.
channel_shift_range = 0.0 # Float. Faixa para mudanças aleatórias dos canais.
cval = 0.0 # Float or Int. Valor usado para pontos fora dos limites quando fill_mode = "constant".
data_format = None # Formato dos dados da imagem, pode ser "channels_first" ou "channels_last". Padrão é "channels_last".
dtype = 'float32' # Dtype para usar nos arrays gerados.
featurewise_center = False # Boolean. Set input mean to 0 over the dataset, feature-wise.
featurewise_std_normalization = False # Boolean. Divide entradas pelo desvio padrão (std) do dataset em recursos, feature-wise.
fill_mode = 'nearest' # Opções: {"constant", "nearest", "reflect" or "wrap"}. Padrão é 'nearest'.
horizontal_flip = False # Boolean. Aplica inversão horizontal nas entragas aleatoriamente.
preprocessing_function = None # Funções que serão aplicadas em cada entrada. A função irá rodar após a imagem ser aumentada e redimensionada.
rescale = 1./255 # Fator de re-escala. Padrão é None. Se None ou 0, nenhuma re-escala serã aplicada, caso contrário cada data será multiplicado pelo valor especificado.
rotation_range = 0 # Int. Faixa de graus para rotações aletórias.
samplewise_center = False # Boolean. Define cada média de amostra para 0.
samplewise_std_normalization = False # Boolean. Divide cada entrada por seu desvio padrão.
shear_range = 0.0 # Float. Intensidade de corte (Ângulo de corte na direção anti-horária em graus) 
vertical_flip = False # Boolean.  Aplica inversão vertical nas entragas aleatoriamente.
zca_whitening = False # Boolean. Aplica ZCA whitening.
zca_epsilon = 1e-06 # epsilon for ZCA whitening. Default é 1e-6.
zoom_range = 0.0 # Float or [lower, upper]. Faixa para zoom aleatório.
'''
width_shift_range or height_shift_range: Float, 1-D array-like or int. 
- Float: uma fração da largura (altura). Se < 1, ou pixels se >= 1. 
- 1-D array-like: elementos aleatórios do array. 
- int: número inteiro de pixels do intervalo (-shift_range, +shift_range)
'''
height_shift_range = 0.0
width_shift_range = 0.0

In [ ]:
!pip install autokeras==1.0.12

In [ ]:
import os
import zipfile

root_folder_name = dataset.split("/")[-1].split(".")[0]
root_folder = os.path.join("/tmp/data", root_folder_name)
with zipfile.ZipFile(dataset, 'r') as zip_ref:
    zip_ref.extractall(root_folder)

#### Criação do iterador de dados para a fase de treinamento

In [ ]:
import tensorflow as tf

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(root_folder, 'train'),
    labels='inferred', label_mode='int',
    batch_size=batch_size)

print(train_data.class_names)

#### Instanciação do [AutoKeras](https://autokeras.com/tutorial/image_regression/) para Regressão de Imagens e busca pela melhor arquitetura e hyperparâmetros

In [ ]:
import autokeras as ak

model = ak.ImageClassifier(
    num_classes=len(train_data.class_names),
    max_trials=trials,
    metrics="accuracy",
    objective="val_loss",
    overwrite=True)

In [ ]:
'''
A busca por arquiteturas é feita utilizando apenas metade 
do número total de épocas. Após selecionada a melhor arquitetura 
e hyperparametros, um novo treinamento dessa arquitetura é realizado
utilizando o número total de épocas e as técnicas de aumentação de
dados escolhidas para obter o modelo com melhor performance.
'''

model.fit(
    train_data,
    validation_split=validation,
    epochs=int(num_epochs/2))

#### Exportação do melhor modelo e exposição da sua configuração de camadas

In [ ]:
model_exported = model.export_model()
model_exported.summary()
model_path = "/tmp/data/model_weights.h5"
model_exported.save(model_path)

#### Criação dos dataloaders para última fase de treino utilizando aumentação

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    brightness_range = brightness_range,
    channel_shift_range = channel_shift_range,
    cval = cval,
    data_format = data_format,
    dtype = dtype,
    featurewise_center = featurewise_center,
    featurewise_std_normalization = featurewise_std_normalization,
    fill_mode = fill_mode,
    horizontal_flip = horizontal_flip,
    preprocessing_function = preprocessing_function,
    rescale = rescale,
    rotation_range = rotation_range,
    samplewise_center = False,
    samplewise_std_normalization = samplewise_std_normalization,
    shear_range = shear_range,
    validation_split = validation_split,
    vertical_flip = vertical_flip,
    zca_whitening = zca_whitening,
    zca_epsilon = zca_epsilon,
    zoom_range = zoom_range,
    height_shift_range = height_shift_range,
    width_shift_range = width_shift_range
)

train_generator = image_generator.flow_from_directory(
    os.path.join(root_folder, 'train'),
    batch_size=batch_size,
    target_size=target_size)

test_datagen = ImageDataGenerator(
    rescale=rescale,
    dtype=dtype)

test_generator = test_datagen.flow_from_directory(
    os.path.join(root_folder, 'test'),
    batch_size=batch_size,
    target_size=target_size)

#### Treino do melhor modelo encontrado

In [ ]:
model = tf.keras.models.load_model(model_path)
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_split=validation,
    verbose=True)

#### Avaliação do melhor modelo no conjunto de teste

In [ ]:
# evaluate the best model
loss, acc = model.evaluate(x=test_generator, verbose=True)
print("Loss: {0} / Acc: {1}".format(loss, acc))

In [ ]:
import numpy as np

predictions = model.predict(x=test_generator)
preds = []
for prediction in predictions:
    indx = np.argmax(prediction)
    preds.append([indx])

#### Geração do relatório de classificação e da matrix de confusão 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

report = classification_report(
    test_generator.classes,
    preds,
    target_names=test_generator.class_indices.keys())
print(report)

In [ ]:
c_matrix = confusion_matrix(test_generator.classes, preds)
print("### Confusion matrix:###\n", c_matrix)

## Salva resultados da tarefa

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.<br>
Use essa pasta para salvar modelos, metadados e outros resultados.

In [ ]:
from joblib import dump

artifacts = {
    "model_path": model_path,
    "class_names": test_generator.classes,
}
dump(artifacts, "/tmp/data/model.joblib")